# Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterSampler

# Loading the Dataset

In [2]:
def load_data(movies_path, ratings_path):
  movies = pd.read_csv(movies_path)
  ratings = pd.read_csv(ratings_path)
  return movies, ratings

movies_path = 'https://raw.githubusercontent.com/Bansal0527/Movie-Recomendation-System/master/Dataset/movies.csv'
ratings_path = 'https://raw.githubusercontent.com/Bansal0527/Movie-Recomendation-System/master/Dataset/ratings.csv'
movies, ratings = load_data(movies_path, ratings_path)

# Preprocessing the Dataset

In [3]:
param_space = {
    'min_user_votes': range(10, 101),
    'min_movie_votes': range(30, 101)
}
n_iter = 100
best_score = float('inf')
best_params = None
param_sampler = ParameterSampler(param_space, n_iter=n_iter, random_state=42)
for params in param_sampler:
  filtered_ratings = ratings.copy()
  user_counts = filtered_ratings['userId'].value_counts()
  movie_counts = filtered_ratings['movieId'].value_counts()
  filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(user_counts[user_counts >= params['min_user_votes']].index)]
  filtered_ratings = filtered_ratings[filtered_ratings['movieId'].isin(movie_counts[movie_counts >= params['min_movie_votes']].index)]
  sparsity = 1 - len(filtered_ratings) / (len(ratings) * len(movies))
  if sparsity < best_score:
    best_score = sparsity
    best_params = params

print("Best parameters:", best_params)
print("Best sparsity score:", best_score)

Best parameters: {'min_user_votes': 15, 'min_movie_votes': 31}
Best sparsity score: 0.9999416111038673


In [4]:
def preprocess_data(ratings, min_user_votes = 15, min_movie_votes = 31):
  user_counts = ratings['userId'].value_counts()
  movie_counts = ratings['movieId'].value_counts()
    
  ratings = ratings[ratings['userId'].isin(user_counts[user_counts >= min_user_votes].index)]
  ratings = ratings[ratings['movieId'].isin(movie_counts[movie_counts >= min_movie_votes].index)]
    
  final_dataset = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
  return final_dataset

final_dataset = preprocess_data(ratings)
movies['genres'] = movies['genres'].str.replace('|', ' ')

# Training the RBF Kernel SVM

This block initializes a TF-IDF vectorizer to convert movie genres into numerical features. Then, it trains a Support Vector Machine (SVM) classifier with a radial basis function (RBF) kernel using the TF-IDF matrix and movie titles, enabling recommendations based on genre similarity.

In [5]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])

svm_model_rbf = SVC(kernel='rbf')
svm_model_rbf.fit(tfidf_matrix, movies['title'])

SVC()

# Movie Recommendation System

Movie recommendation on the basis of the model trained

In [6]:
def get_movie_recommendation_svm_rbf(movie_name, model, vectorizer, movies_df, k=10):
  movie_idx = movies_df[movies_df['title'].str.contains(movie_name, case=False)].index
  if len(movie_idx) > 0:
    movie_features = vectorizer.transform(movies_df.iloc[movie_idx]['genres'])
    similarity_scores = cosine_similarity(movie_features, tfidf_matrix)
    similar_movies_idx = np.argsort(similarity_scores[0])[::-1][1:k+1] 
    recommendations = movies_df.iloc[similar_movies_idx][['title', 'genres']]
    return recommendations
  else:
    return "No movies found. Please check your input."

input_movie = "Memento"
recommendations_rbf = get_movie_recommendation_svm_rbf(input_movie, svm_model_rbf, tfidf_vectorizer, movies)
print(recommendations_rbf)

                                       title                   genres
8692                    Black Panther (2017)  Action Adventure Sci-Fi
8425       X-Men: Days of Future Past (2014)  Action Adventure Sci-Fi
4619              Returner (Ritaanaa) (2002)  Action Adventure Sci-Fi
8458  Transformers: Age of Extinction (2014)  Action Adventure Sci-Fi
176                        Waterworld (1995)  Action Adventure Sci-Fi
6863     20,000 Leagues Under the Sea (1916)  Action Adventure Sci-Fi
8475          Guardians of the Galaxy (2014)  Action Adventure Sci-Fi
9484                    Power Rangers (2017)  Action Adventure Sci-Fi
8489                Jupiter Ascending (2015)  Action Adventure Sci-Fi
4306                 Bulletproof Monk (2003)  Action Adventure Sci-Fi
